In [2]:
import pandas as pd

# Assuming this function needs to be defined:
def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5/9

class CornGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # Convert the temperature to Celsius if required
        if not self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(fahrenheit_to_celsius)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(fahrenheit_to_celsius)
    
    def con_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 10 if self.celsius else 50)
            max_temp = min(row[self.max_temp_column], 30 if self.celsius else 86)

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 10 if self.celsius else 50
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

In [3]:
import pandas as pd

Wadena = pd.read_csv("Wadena_County_NOAA_nClimGrid.csv")
Cass = pd.read_csv("Cass_County_NOAA_nClimGrid.csv")
Crow_Wing = pd.read_csv("Crow_Wing_County_NOAA_nClimGrid.csv")
Todd = pd.read_csv("Todd_County_NOAA_nClimGrid.csv")
Morrison = pd.read_csv("Morrison_County_NOAA_nClimGrid.csv")
Douglas = pd.read_csv("Douglas_County_NOAA_nClimGrid.csv")
Otter_Tail = pd.read_csv("Otter_Tail_County_NOAA_nClimGrid.csv")
Pope = pd.read_csv("Pope_County_NOAA_nClimGrid.csv")
Stearns = pd.read_csv("Stearns_County_NOAA_nClimGrid.csv")
Sherburne = pd.read_csv("Sherburne_County_NOAA_nClimGrid.csv")
Benton = pd.read_csv("Benton_County_NOAA_nClimGrid.csv")
Becker = pd.read_csv("Becker_County_NOAA_nClimGrid.csv")
Hubbard= pd.read_csv("Hubbard_County_NOAA_nClimGrid.csv")
Kandiyohi= pd.read_csv("Kandiyohi_County_NOAA_nClimGrid.csv")



In [ ]:
import pandas as pd

# Dictionary of all counties
# list of county DataFrames
county_dfs = {
    "Wadena": Wadena,
    "Cass": Cass,
    "Crow_Wing": Crow_Wing,
    "Todd": Todd,
    "Morrison": Morrison,
    "Douglas": Douglas,
    "Otter_Tail": Otter_Tail,
    "Pope": Pope,
    "Stearns": Stearns,
    "Sherburne": Sherburne,
    "Benton": Benton,
    "Becker": Becker,
    "Hubbard": Hubbard,
    "Kandiyohi": Kandiyohi
}
# Loop through each county
for county_name, df in county_dfs.items():
    df['time'] = pd.to_datetime(df['time'])
    all_years = []

    for year in range(1951, 2025):
        corn = CornGDD(df, 'time', 'tmin', 'tmax')
        start = pd.to_datetime(f"{year}-01-01").date()
        end = pd.to_datetime(f"{year}-12-31").date()

        gdd_df = corn.con_gdd(start, end)
        gdd_df['Year'] = year
        gdd_df['Cumulative_GDD'] = gdd_df['Cumulative_GDD'].apply(c_to_f)

        all_years.append(gdd_df)

    # Combine and save
    historical_df = pd.concat(all_years, ignore_index=True)
    historical_df.to_csv(f"{county_name}_Corn_GDD_1951_2024.csv", index=False)
    print(f"Saved: {county_name}_Corn_GDD_1951_2024.csv")


In [5]:
import pandas as pd

# Function to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(c):
    return (c * 9/5)

# Dictionary of county DataFrames
county_dfs = {
    "Wadena": Wadena,
    "Cass": Cass,
    "Crow_Wing": Crow_Wing,
    "Todd": Todd,
    "Morrison": Morrison,
    "Douglas": Douglas,
    "Otter_Tail": Otter_Tail,
    "Pope": Pope,
    "Stearns": Stearns,
    "Sherburne": Sherburne,
    "Benton": Benton,
    "Becker": Becker,
    "Hubbard": Hubbard,
    "Kandiyohi": Kandiyohi
}

# Loop through each county
for county_name, df in county_dfs.items():
    df['time'] = pd.to_datetime(df['time'])
    all_years = []

    for year in range(1951, 2025):
        corn = CornGDD(df, 'time', 'tmin', 'tmax')  # Output in Celsius
        start = pd.to_datetime(f"{year}-04-15").date()
        end = pd.to_datetime(f"{year}-12-31").date()

        gdd_df = corn.con_gdd(start, end)
        gdd_df['Year'] = year

        # Manually convert GDD to Fahrenheit
        gdd_df['GDD'] = gdd_df['GDD'].apply(celsius_to_fahrenheit)
        gdd_df['Cumulative_GDD'] = gdd_df['GDD'].cumsum()

        all_years.append(gdd_df)

    historical_df = pd.concat(all_years, ignore_index=True)
    historical_df.to_csv(f"{county_name}_growing2024_Corn_GDD_1951_2024.csv", index=False)
    print(f"Saved: {county_name}_growing2024_Corn_GDD_1951_2024.csv")


Saved: Wadena_growing2024_Corn_GDD_1951_2024.csv
Saved: Cass_growing2024_Corn_GDD_1951_2024.csv
Saved: Crow_Wing_growing2024_Corn_GDD_1951_2024.csv
Saved: Todd_growing2024_Corn_GDD_1951_2024.csv
Saved: Morrison_growing2024_Corn_GDD_1951_2024.csv
Saved: Douglas_growing2024_Corn_GDD_1951_2024.csv
Saved: Otter_Tail_growing2024_Corn_GDD_1951_2024.csv
Saved: Pope_growing2024_Corn_GDD_1951_2024.csv
Saved: Stearns_growing2024_Corn_GDD_1951_2024.csv
Saved: Sherburne_growing2024_Corn_GDD_1951_2024.csv
Saved: Benton_growing2024_Corn_GDD_1951_2024.csv
Saved: Becker_growing2024_Corn_GDD_1951_2024.csv
Saved: Hubbard_growing2024_Corn_GDD_1951_2024.csv
Saved: Kandiyohi_growing2024_Corn_GDD_1951_2024.csv


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Function to convert Celsius GDD to Fahrenheit GDD
def celsius_to_fahrenheit(c):
    return c * 9 / 5

# Function to get list of planting dates
def generate_planting_dates(start_date, end_date):
    return [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Dictionary of your county dataframes (make sure they are defined properly)
county_dfs = {
    "Wadena": Wadena,
    "Cass": Cass,
    "Crow_Wing": Crow_Wing,
    "Todd": Todd,
    "Morrison": Morrison,
    "Douglas": Douglas,
    "Otter_Tail": Otter_Tail,
    "Pope": Pope,
    "Stearns": Stearns,
    "Sherburne": Sherburne,
    "Benton": Benton,
    "Becker": Becker,
    "Hubbard": Hubbard,
    "Kandiyohi": Kandiyohi
}

# Loop through each county and year
for county_name, df in county_dfs.items():
    df['time'] = pd.to_datetime(df['time'])

    all_planting_results = []
    for year in range(1951, 2025):
        corn = CornGDD(df, 'time', 'tmin', 'tmax')  # Your existing GDD class
        start_year = pd.to_datetime(f"{year}-04-15")
        end_year = pd.to_datetime(f"{year}-12-31")
        full_gdd_df = corn.con_gdd(start_year.date(), end_year.date())
        full_gdd_df['time'] = pd.to_datetime(full_gdd_df['time'])
        full_gdd_df['GDD'] = celsius_to_fahrenheit(full_gdd_df['GDD'])

        planting_dates = generate_planting_dates(pd.to_datetime(f"{year}-04-15"), pd.to_datetime(f"{year}-06-15"))

        for planting_date in planting_dates:
            sub_df = full_gdd_df[full_gdd_df['time'] >= planting_date].copy()
            sub_df['Year'] = year
            sub_df['Planting Date'] = planting_date.date()
            sub_df['DAP'] = (sub_df['time'] - planting_date).dt.days
            sub_df['Cumulative_GDD'] = sub_df['GDD'].cumsum()
            all_planting_results.append(sub_df)

    historical_df = pd.concat(all_planting_results, ignore_index=True)
    output_name = f"{county_name}_AllPlantingDates_Corn_GDD_1951_2024.csv"
    historical_df.to_csv(output_name, index=False)
    print(f"Saved: {output_name}")


In [ ]:
import pandas as pd

# Load datasets
becker = pd.read_csv("Becker 2024 corn Growth staging.csv")
wadena = pd.read_csv("Wadena_AllPlantingDates_Corn_GDD_1951_2024.csv")

# Clean up
becker.columns = becker.columns.str.strip()
becker["Date"] = pd.to_datetime(becker["Date"] + "-2024", format="%d-%b-%Y", errors="coerce")
wadena["time"] = pd.to_datetime(wadena["time"])
wadena["Planting Date"] = pd.to_datetime(wadena["Planting Date"])

# Becker AGDD thresholds (mode per stage)
stages = ['VE', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6']
thresholds = (becker
              .groupby("Growth Stage")["AGDD"]
              .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
              .dropna()
              .loc[stages]
              .reset_index())
thresholds.columns = ["Stage", "AGDD Threshold"]

# Process by (Year, Planting Date)
results = []
for (year, planting), group in wadena.groupby(["Year", "Planting Date"]):
    group = group[group["time"] >= planting].copy()
    group = group.sort_values("time")
    group["Cumulative_GDD"] = group["GDD"].cumsum()

    for _, row in thresholds.iterrows():
        stage = row["Stage"]
        agdd_thresh = row["AGDD Threshold"]
        met = group[group["Cumulative_GDD"] >= agdd_thresh]

        if not met.empty:
            first_hit = met.iloc[0]
            dap = (first_hit["time"] - planting).days
            results.append({
                "Crop": "Corn",
                "County": "Wadena",
                "Year": year,
                "Planting Date": planting.date(),
                "Stage": stage,
                "AGDD Threshold (°F)": round(agdd_thresh, 2),
                "DAP Reached": dap,
                "AGDD Reached (°F)": round(first_hit["Cumulative_GDD"], 1)
            })
        else:
            results.append({
                "Crop": "Corn",
                "County": "Wadena",
                "Year": year,
                "Planting Date": planting.date(),
                "Stage": stage,
                "AGDD Threshold (°F)": round(agdd_thresh, 2),
                "DAP Reached": "",
                "AGDD Reached (°F)": ""
            })

# Save results
final_df = pd.DataFrame(results)
final_df.to_csv("Wadena_Corn_GrowthStages_1951_2022_AllPlantingDates.csv", index=False)
print("File saved.")


In [ ]:
import pandas as pd
import os

# List of all counties
counties = [
    "Wadena", "Cass", "Crow_Wing", "Todd", "Morrison", "Douglas",
    "Otter_Tail", "Pope", "Stearns", "Sherburne", "Benton",
     "Hubbard", "Kandiyohi"
]

# Load Becker 2024 growth stages
becker = pd.read_csv("Becker 2024 corn Growth staging.csv")
becker.columns = becker.columns.str.strip()
becker["Date"] = pd.to_datetime(becker["Date"] + "-2024", format="%d-%b-%Y", errors="coerce")

# Extract mode AGDD threshold per stage
stages = ['VE', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6']
thresholds = (becker
              .groupby("Growth Stage")["AGDD"]
              .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
              .dropna()
              .loc[stages]
              .reset_index())
thresholds.columns = ["Stage", "AGDD Threshold"]

# Output folder
os.makedirs("AllCounties_CornGrowthStages_1951_2022", exist_ok=True)

# Process each county
for county in counties:
    input_file = f"{county}_AllPlantingDates_Corn_GDD_1951_2024.csv"
    if not os.path.exists(input_file):
        print(f"⚠️ File not found: {input_file}")
        continue

    df = pd.read_csv(input_file)
    df["time"] = pd.to_datetime(df["time"])
    df["Planting Date"] = pd.to_datetime(df["Planting Date"])

    results = []
    for (year, planting), group in df.groupby(["Year", "Planting Date"]):
        group = group[group["time"] >= planting].copy()
        group = group.sort_values("time")
        group["Cumulative_GDD"] = group["GDD"].cumsum()

        for _, row in thresholds.iterrows():
            stage = row["Stage"]
            agdd_thresh = row["AGDD Threshold"]
            met = group[group["Cumulative_GDD"] >= agdd_thresh]

            if not met.empty:
                first_hit = met.iloc[0]
                dap = (first_hit["time"] - planting).days
                results.append({
                    "Crop": "Corn",
                    "County": county,
                    "Year": year,
                    "Planting Date": planting.date(),
                    "Stage": stage,
                    "AGDD Threshold (°F)": round(agdd_thresh, 2),
                    "DAP Reached": dap,
                    "AGDD Reached (°F)": round(first_hit["Cumulative_GDD"], 1)
                })
            else:
                results.append({
                    "Crop": "Corn",
                    "County": county,
                    "Year": year,
                    "Planting Date": planting.date(),
                    "Stage": stage,
                    "AGDD Threshold (°F)": round(agdd_thresh, 2),
                    "DAP Reached": "",
                    "AGDD Reached (°F)": ""
                })

    # Save per county
    out_df = pd.DataFrame(results)
    out_df.to_csv(f"AllCounties_CornGrowthStages_1951_2022/{county}_Corn_GrowthStages_1951_2022.csv", index=False)
    print(f"Saved: {county}_Corn_GrowthStages_1951_2022.csv")


Saved: Wadena_Corn_GrowthStages_1951_2022.csv
Saved: Cass_Corn_GrowthStages_1951_2022.csv
Saved: Crow_Wing_Corn_GrowthStages_1951_2022.csv
Saved: Todd_Corn_GrowthStages_1951_2022.csv
Saved: Morrison_Corn_GrowthStages_1951_2022.csv
Saved: Douglas_Corn_GrowthStages_1951_2022.csv


In [48]:
import pandas as pd
import os

# === Step 0: Setup ===
county_names = [
    "Wadena", "Cass", "Crow_Wing", "Todd", "Morrison", "Douglas",
    "Otter_Tail", "Pope", "Stearns", "Sherburne", "Benton",
    "Becker", "Hubbard", "Kandiyohi"
]
planting_date = pd.Timestamp("2024-05-01")

# Output folder
output_folder = "Corn_2024_Growth_Stages_Diff"
os.makedirs(output_folder, exist_ok=True)

# === Step 1: Load and process Becker 2024 growth stages ===
becker_2024 = pd.read_csv("Becker 2024 corn Growth staging.csv")
becker_2024.columns = becker_2024.columns.str.strip()
becker_2024["Date"] = pd.to_datetime(becker_2024["Date"] + "-2024", format="%d-%b-%Y", errors="coerce")

ordered_stages = [
    'VE', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
    'R1', 'R2', 'R3', 'R4', 'R5', 'R6']

# Mode-based thresholds
becker_mode = becker_2024.groupby("Date")["Growth Stage"].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()

becker_mode_with_agdd = pd.merge(becker_mode, becker_2024, on=["Date", "Growth Stage"], how="left")
becker_mode_with_agdd["Becker DAP"] = (becker_mode_with_agdd["Date"] - planting_date).dt.days
becker_thresholds = becker_mode_with_agdd.drop_duplicates("Growth Stage")[["Growth Stage", "AGDD", "Becker DAP"]]
becker_thresholds["Growth Stage"] = pd.Categorical(becker_thresholds["Growth Stage"], categories=ordered_stages, ordered=True)
becker_thresholds = becker_thresholds.sort_values("Growth Stage").dropna()

# === Step 2: Function to calculate growth stage table with differences ===
def get_stage_table_with_becker_dap(df, thresholds_df, planting_date, county, crop_name="Corn"):
    df["time"] = pd.to_datetime(df["time"])
    df = df[df["Year"] == 2024].copy()
    df["DAP"] = (df["time"] - planting_date).dt.days

    results = []
    for _, row in thresholds_df.iterrows():
        stage = row["Growth Stage"]
        threshold = row["AGDD"]
        dap_becker = int(row["Becker DAP"])
        hit = df[df["Cumulative_GDD"] >= threshold]
        if not hit.empty:
            match = hit.iloc[0]
            county_dap = (match["time"] - planting_date).days
            county_agdd = round(match["Cumulative_GDD"], 1)
            results.append({
                "Crop": crop_name,
                "Stage": stage,
                "Becker DAP": dap_becker,
                f"{county} DAP": county_dap,
                "DAP Difference": county_dap - dap_becker,
                "Becker AGDD (°F)": round(threshold, 2),
                f"{county} AGDD (°F)": county_agdd,
                "AGDD Difference": county_agdd - round(threshold, 2)
            })
    return pd.DataFrame(results)

# === Step 3: Loop through counties and save results ===
for county in county_names:
    file_name = f"{county}_growing2024_Corn_GDD_1951_2024.csv"
    if os.path.isfile(file_name):
        df = pd.read_csv(file_name, parse_dates=["time"])
        final_table = get_stage_table_with_becker_dap(df, becker_thresholds, planting_date, county)

        final_table["Stage"] = pd.Categorical(final_table["Stage"], categories=ordered_stages, ordered=True)
        final_table = final_table.sort_values("Stage").reset_index(drop=True)

        output_path = os.path.join(output_folder, f"{county}_2024_growth_stage_diff.csv")
        final_table.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}")
    else:
        print(f" File not found: {file_name}")


✅ Saved: Corn_2024_Growth_Stages_Diff/Wadena_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Cass_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Crow_Wing_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Todd_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Morrison_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Douglas_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Otter_Tail_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Pope_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Stearns_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Sherburne_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Benton_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Becker_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Hubbard_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Kandiyohi_2024_gr

In [19]:
import pandas as pd
import os

# === Step 0: Setup ===
county_names = [
    "Wadena", "Cass", "Crow_Wing", "Todd", "Morrison", "Douglas",
    "Otter_Tail", "Pope", "Stearns", "Sherburne", "Benton",
    "Becker", "Hubbard", "Kandiyohi"
]
planting_date = pd.Timestamp("2024-05-01")

# Output folder
output_folder = "Corn_2024_Growth_Stages_Diff"
os.makedirs(output_folder, exist_ok=True)

# === Step 1: Load and process Becker 2024 growth stages ===
becker_2024 = pd.read_csv("Becker 2024 corn Growth staging.csv")
becker_2024.columns = becker_2024.columns.str.strip()
becker_2024["Date"] = pd.to_datetime(becker_2024["Date"] + "-2024", format="%d-%b-%Y", errors="coerce")

ordered_stages = [
    'VE', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
    'R1', 'R2', 'R3', 'R4', 'R5', 'R6'
]

# Mode-based thresholds
becker_mode = becker_2024.groupby("Date")["Growth Stage"].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()

becker_mode_with_agdd = pd.merge(becker_mode, becker_2024, on=["Date", "Growth Stage"], how="left")
becker_mode_with_agdd["Becker DAP"] = (becker_mode_with_agdd["Date"] - planting_date).dt.days
becker_thresholds = becker_mode_with_agdd.drop_duplicates("Growth Stage")[["Growth Stage", "AGDD", "Becker DAP"]]
becker_thresholds["Growth Stage"] = pd.Categorical(becker_thresholds["Growth Stage"], categories=ordered_stages, ordered=True)
becker_thresholds = becker_thresholds.sort_values("Growth Stage").dropna()

# === Step 2: Function to calculate growth stage table with differences ===
def get_stage_table_with_becker_dap(df, thresholds_df, planting_date, county, crop_name="Corn"):
    df["time"] = pd.to_datetime(df["time"])
    df = df[df["Year"] == 2024].copy()
    df["DAP"] = (df["time"] - planting_date).dt.days

    results = []
    for _, row in thresholds_df.iterrows():
        stage = row["Growth Stage"]
        threshold = row["AGDD"]
        dap_becker = int(row["Becker DAP"])
        hit = df[df["Cumulative_GDD"] >= threshold]

        if not hit.empty:
            match = hit.iloc[0]
            county_dap = (match["time"] - planting_date).days
            county_agdd = round(match["Cumulative_GDD"], 1)
            dap_diff = county_dap - dap_becker
            agdd_diff = county_agdd - round(threshold, 2)
        else:
            county_dap = None
            county_agdd = None
            dap_diff = None
            agdd_diff = None

        results.append({
            "Crop": crop_name,
            "Stage": stage,
            "Becker DAP": dap_becker,
            f"{county} DAP": county_dap,
            "DAP Difference": dap_diff,
            "Becker AGDD (°F)": round(threshold, 2),
            f"{county} AGDD (°F)": county_agdd,
            "AGDD Difference": agdd_diff
        })

    return pd.DataFrame(results)

# === Step 3: Loop through counties and save results ===
for county in county_names:
    file_name = f"{county}_growing2024_Corn_GDD_1951_2024.csv"
    if os.path.isfile(file_name):
        df = pd.read_csv(file_name, parse_dates=["time"])
        final_table = get_stage_table_with_becker_dap(df, becker_thresholds, planting_date, county)

        final_table["Stage"] = pd.Categorical(final_table["Stage"], categories=ordered_stages, ordered=True)
        final_table = final_table.sort_values("Stage").reset_index(drop=True)

        output_path = os.path.join(output_folder, f"{county}_2024_growth_stage_diff.csv")
        final_table.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}")
    else:
        print(f"⚠️ File not found: {file_name}")


✅ Saved: Corn_2024_Growth_Stages_Diff/Wadena_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Cass_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Crow_Wing_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Todd_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Morrison_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Douglas_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Otter_Tail_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Pope_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Stearns_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Sherburne_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Benton_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Becker_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Hubbard_2024_growth_stage_diff.csv
✅ Saved: Corn_2024_Growth_Stages_Diff/Kandiyohi_2024_gr